# Demo notebook for model learning using scikit-learn
We use the journal ranking dataset to learn correlations in the data using linear regression. See also [here](https://scikit-learn.org/stable/auto_examples/inspection/plot_linear_model_coefficient_interpretation.html#sphx-glr-auto-examples-inspection-plot-linear-model-coefficient-interpretation-py) and [here](https://scikit-learn.org/stable/common_pitfalls.html) for references.

In [ ]:
# import at the top of the notebook
from sklearn import linear_model
from sklearn.model_selection import (
    train_test_split,
    cross_validate,
    RepeatedKFold,
    cross_val_predict,
)
from sklearn.compose import TransformedTargetRegressor, make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    PredictionErrorDisplay,
    mean_absolute_error,
    median_absolute_error,
)
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# pre-processing steps as per the data notebook
raw_dataset = pd.read_csv("../data/journal_ranking_data.csv")
clean_dataset = raw_dataset.drop(raw_dataset[raw_dataset["CiteScore"].gt(100)].index)
clean_dataset = clean_dataset.drop(
    clean_dataset[clean_dataset["Cites/Doc. 2y"].gt(43)].index
)
clean_dataset = clean_dataset.dropna()
clean_dataset = clean_dataset.drop_duplicates(subset=["CiteScore", "Cites/Doc. 2y"])

In [ ]:
fig = px.histogram(clean_dataset, x="CiteScore", nbins=400)
fig.show()

In [ ]:
fig = px.histogram(clean_dataset, x="Cites/Doc. 2y", nbins=400)
fig.show()

In [ ]:
# create train and test set
train, test = train_test_split(clean_dataset, test_size=0.2)

In [ ]:
# export the two columns to np arrays
X = train["Cites/Doc. 2y"].to_numpy().reshape(-1, 1)
y = train["CiteScore"].to_numpy().reshape(-1, 1)
X_test = test["Cites/Doc. 2y"].to_numpy().reshape(-1, 1)
y_test = test["CiteScore"].to_numpy().reshape(-1, 1)

In [ ]:
# fit a linear regression model
reg = linear_model.LinearRegression()

In [ ]:
reg.fit(X, y)

In [ ]:
# check the coefficients
reg.coef_, reg.intercept_

In [ ]:
# show the linear relationship using the predicted coefficients
xvals = np.arange(0, 60, 1)
yvals = reg.predict(xvals.reshape(-1, 1))
yvals[:, 0]

In [ ]:
# plot the model
fig = px.scatter(train, x="Cites/Doc. 2y", y="CiteScore")
fig.add_trace(go.Scatter(x=xvals, y=yvals[:, 0]))
fig.show()

In [ ]:
sns.pairplot(train[["Cites/Doc. 2y", "CiteScore"]], kind="reg", diag_kind="kde")

In [ ]:
mae_train = mean_absolute_error(y, reg.predict(X))
y_test_predict = reg.predict(X_test)
mae_test = mean_absolute_error(y_test, y_test_predict)

In [ ]:
mae_train

In [ ]:
mae_test

In [ ]:
scores = {
    "MAE on training set": f"{mae_train:.1f}",
    "MAE on testing set": f"{mae_test:.1f}",
}

In [ ]:
_, ax = plt.subplots(figsize=(5, 5))
display = PredictionErrorDisplay.from_predictions(
    y_test,
    y_test_predict,
    kind="actual_vs_predicted",
    ax=ax,
    scatter_kwargs={"alpha": 0.5},
)
ax.set_title("Linear")
for name, score in scores.items():
    ax.plot([], [], " ", label=f"{name}: {score}")
ax.legend(loc="upper left")
plt.tight_layout()

# Now include more features in the fitting

In [ ]:
clean_dataset.head()

In [ ]:
# select features for the training
selected_columns = [
    "SJR-index",
    "H-index",
    "Total Docs.",
    "Total Docs. 3y",
    "Total Refs.",
    "Total Cites 3y",
    "Citable Docs. 3y",
    "Cites/Doc. 2y",
    "Refs./Doc.",
]
multiple_features = clean_dataset[selected_columns + ["CiteScore"]].copy()
# drop duplicates
multiple_features = multiple_features.drop_duplicates()
train, test = train_test_split(multiple_features, test_size=0.2)

In [ ]:
train.info()
# we only have to deal with numerical data and do not need to map any categorical data

In [ ]:
# specify which features should be trained for and which should be predicted
X = train[selected_columns]
y = train["CiteScore"]
X_test = test[selected_columns]
y_test = test["CiteScore"]

In [ ]:
sns.pairplot(train, kind="reg", diag_kind="kde")
plt.show()
# distribution for CiteScore has a long tail, could take the log to
# approximate a normal distribution
# also shows clearly which values are correlated by linear relationship
# as we saw previously, Cites/Doc. 2y is linearly correlated with CiteScore

In [ ]:
# scale the numerical columns for a more balanced representation in the model weights
scale_columns = selected_columns

preprocessor = make_column_transformer(
    (StandardScaler(), scale_columns),
)

In [ ]:
# invoke the model architecture: a linear model using ridge regression and a very small
# regularization parameter
model = make_pipeline(
    preprocessor,
    TransformedTargetRegressor(
        regressor=linear_model.Ridge(alpha=1e-10),
    ),
)
model.fit(X, y)

In [ ]:
# call the fit method on the model
model.fit(X, y)

In [ ]:
# investigate the model performance
mae_train = median_absolute_error(y, model.predict(X))
y_pred = model.predict(X_test)
mae_test = median_absolute_error(y_test, y_pred)
scores = {
    "MedAE on training set": f"{mae_train:.2f} CiteScore",
    "MedAE on testing set": f"{mae_test:.2f} CiteScore",
}

In [ ]:
print(scores)

In [ ]:
_, ax = plt.subplots(figsize=(5, 5))
display = PredictionErrorDisplay.from_predictions(
    y_test, y_pred, kind="actual_vs_predicted", ax=ax, scatter_kwargs={"alpha": 0.5}
)
ax.set_title("Ridge model, small regularization")
for name, score in scores.items():
    ax.plot([], [], " ", label=f"{name}: {score}")
ax.legend(loc="upper left")
plt.tight_layout()

- Moderate performance as the predicted values lie around the mark of the actual values. 
- The model is not overfitting as the performance on the test set is similar to the training set. 
- The model is not underfitting as the performance is not significantly worse than the training set. 

In [ ]:
feature_names = model[:-1].get_feature_names_out()


coefs = pd.DataFrame(
    model[-1].regressor_.coef_,
    columns=["Coefficients importance"],
    index=feature_names,
)
coefs.plot.barh(figsize=(9, 7))
plt.title("Ridge model, small regularization, normalized variables")
plt.xlabel("Raw coefficient values")
plt.axvline(x=0, color=".5")
plt.subplots_adjust(left=0.3)

- The values of the coefficients seem to indicate that Cites/Docs. 2y, Citable Docs. 3y and Total Docs. 3y have most significance on the prediced value

In [ ]:
# retrain using cross-validation
cv = RepeatedKFold(n_splits=5, n_repeats=5, random_state=0)
cv_model = cross_validate(
    model,
    X,
    y,
    cv=cv,
    return_estimator=True,
    n_jobs=2,
    scoring=["neg_mean_absolute_error"],
)
coefs = pd.DataFrame(
    [est[-1].regressor_.coef_ for est in cv_model["estimator"]], columns=feature_names
)

In [ ]:
plt.figure(figsize=(9, 7))
sns.stripplot(data=coefs, orient="h", palette="dark:k", alpha=0.5)
sns.boxplot(data=coefs, orient="h", color="cyan", saturation=0.5, whis=10)
plt.axvline(x=0, color=".5")
plt.title("Coefficient variability")
plt.subplots_adjust(left=0.3)
plt.savefig("coeff_variability.png")
plt.show()

Coefficient importance and variability over several folds: The features Citable Docs. 3y and Total Docs. 3y show vastly changing coefficients. This is due to their interdependence: Their effect on the value to be predicted is difficult to treat separately (collinear features).

In [ ]:
print(cv_model.keys())

In [ ]:
mae_train_average = abs(np.mean(cv_model["test_neg_mean_absolute_error"]))

In [ ]:
# use cross validation to predict the test set
# this is usually not done, but for demonstration purposes
y_pred = cross_val_predict(model, X_test, y_test, cv=3)

In [ ]:
mae_test = mean_absolute_error(y_test, y_pred)
scores = {
    "MAE on training set": f"{mae_train_average:.2f} CiteScore",
    "MAE on testing set": f"{mae_test:.2f} CiteScore",
}
print(scores)

In [ ]:
_, ax = plt.subplots(figsize=(5, 5))
display = PredictionErrorDisplay.from_predictions(
    y_test, y_pred, kind="actual_vs_predicted", ax=ax, scatter_kwargs={"alpha": 0.5}
)
ax.set_title("Ridge model, small regularization, across five folds")
for name, score in scores.items():
    ax.plot([], [], " ", label=f"{name}: {score}")
ax.legend(loc="upper left")
plt.tight_layout()